In [ ]:
import pandas as pd

df = pd.read_csv("data/all_data_merged.csv", parse_dates=["DATE"])

df = df.sort_values(["DISTRICT","DATE"])

district_groups = {d: g.reset_index(drop=True)
                   for d, g in df.groupby("DISTRICT")}

print(f"Number of districts: {len(district_groups)}")
print(f"Districts: {list(district_groups.keys())}")
print(f"First district:\n{district_groups[1].head()}")
print("shape:", district_groups[1].shape)

Number of districts: 37
Districts: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]
First district:
        DATE  DISTRICT  WEATHER_CONDITION  SPECIAL_DAY  DAY_OF_WEEK_NUM  \
0 2024-01-01         1                  1            1                1   
1 2024-01-03         1                  2            0                3   
2 2024-01-04         1                  2            0                4   
3 2024-01-05         1                  2            0                5   
4 2024-01-06         1                  1            0                6   

   POPULATION  RAILWAY_STATIONS  BUS_STATIONS  CONGESTION  
0      344868                 5           678       0.213  
1      344868                 5           678       0.243  
2      344868                 5           678       0.234  
3      344868                 5           678       0.238  
4      344868                 5           678       0.246 

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def make_sequences(group, seq_len=10):
    X_raw = group[["WEATHER_CONDITION","SPECIAL_DAY","DAY_OF_WEEK_NUM",
                   "POPULATION","RAILWAY_STATIONS","BUS_STATIONS"]].values
    y_raw = group["CONGESTION"].values.reshape(-1,1)
    
    sx, sy = MinMaxScaler(), MinMaxScaler()
    Xs, ys = sx.fit_transform(X_raw), sy.fit_transform(y_raw)
    
    Xs_seq, ys_seq = [], []
    for i in range(len(Xs) - seq_len):
        Xs_seq.append(Xs[i:i+seq_len])
        ys_seq.append(ys[i+seq_len])
    return np.array(Xs_seq), np.array(ys_seq), sy

In [ ]:
import torch
import torch.nn as nn

class LSTMRegressor(nn.Module):
    def __init__(self, n_features, hidden_dim=32, n_layers=1):
        super().__init__()
        self.lstm = nn.LSTM(n_features, hidden_dim, n_layers, batch_first=True)
        self.out = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        h, _ = self.lstm(x)
        return self.out(h[:, -1, :])

In [ ]:
def flatten_weights(model):
    return np.concatenate([p.detach().cpu().numpy().ravel() for p in model.parameters()])

def unflatten_weights(model, vec):
    idx = 0
    for p in model.parameters():
        numel = p.numel()
        chunk = vec[idx:idx+numel].reshape(p.shape)
        p.data.copy_(torch.from_numpy(chunk))
        idx += numel

In [ ]:
from pyswarms.single import GlobalBestPSO

def make_fitness(X_train, y_train, device="cpu"):
    model = LSTMRegressor(n_features=X_train.shape[-1]).to(device)
    dim = sum(p.numel() for p in model.parameters())

    def fitness(pop):
        losses = []
        criterion = nn.MSELoss()
        Xb = torch.tensor(X_train, dtype=torch.float32).to(device)
        yb = torch.tensor(y_train, dtype=torch.float32).to(device)
        for particle in pop:
            unflatten_weights(model, particle)
            preds = model(Xb).squeeze()
            loss = criterion(preds, yb.squeeze()).item()
            losses.append(loss)
        return np.array(losses)

    return fitness, dim, model

In [ ]:
import numpy as np

def pso_optimize(
    fitness_func,
    dim,
    n_particles=30,
    n_iters=100,
    w=0.9,
    c1=0.5,
    c2=0.3,
    bounds=None,
    verbose=True
):
    if bounds is not None:
        low, high = bounds
        pos = np.random.uniform(low, high, (n_particles, dim))
    else:
        pos = np.random.randn(n_particles, dim) * 0.1
    vel = np.zeros_like(pos)

    fitness = fitness_func(pos)
    pbest_pos   = pos.copy()
    pbest_score = fitness.copy()
    gbest_idx   = np.argmin(pbest_score)
    gbest_pos   = pbest_pos[gbest_idx].copy()
    gbest_score = pbest_score[gbest_idx]

    for it in range(1, n_iters+1):
        r1 = np.random.rand(n_particles, dim)
        r2 = np.random.rand(n_particles, dim)

        vel = (
            w * vel
            + c1 * r1 * (pbest_pos - pos)
            + c2 * r2 * (gbest_pos - pos)
        )

        pos = pos + vel

        if bounds is not None:
            pos = np.clip(pos, low, high)

        fitness = fitness_func(pos)

        better_mask = fitness < pbest_score
        pbest_pos[better_mask]   = pos[better_mask]
        pbest_score[better_mask] = fitness[better_mask]

        current_best_idx = np.argmin(pbest_score)
        if pbest_score[current_best_idx] < gbest_score:
            gbest_score = pbest_score[current_best_idx]
            gbest_pos   = pbest_pos[current_best_idx].copy()

        if verbose and it % 10 == 0:
            print(f"Iter {it:3d}/{n_iters},  gbest MSE = {gbest_score:.6f}")

    return gbest_score, gbest_pos

In [ ]:
import numpy as np
import torch
import torch.nn as nn

def flatten_gradients(model):
    return np.concatenate([
        p.grad.detach().cpu().numpy().ravel()
        for p in model.parameters()
    ])

def hybrid_pso_optimize(
    model,
    X_train, y_train,
    dim,
    n_particles=30,
    n_iters=100,
    w=0.9,
    c1=0.5,
    c2=0.3,
    alpha=0.01,
    bounds=None,
    verbose=True
):
    criterion = nn.MSELoss()

    if bounds is not None:
        low, high = bounds
        pos = np.random.uniform(low, high, (n_particles, dim))
    else:
        pos = np.random.randn(n_particles, dim)*0.1
    vel = np.zeros_like(pos)

    def eval_fitness(pop):
        out = []
        for p in pop:
            unflatten_weights(model, p)
            with torch.no_grad():
                preds = model(X_train).squeeze()
            loss = criterion(preds, y_train.squeeze()).item()
            out.append(loss)
        return np.array(out)

    fitness = eval_fitness(pos)
    pbest_pos   = pos.copy()
    pbest_score = fitness.copy()
    gidx        = np.argmin(pbest_score)
    gbest_pos   = pbest_pos[gidx].copy()
    gbest_score = pbest_score[gidx]

    for it in range(1, n_iters+1):
        r1 = np.random.rand(n_particles, dim)
        r2 = np.random.rand(n_particles, dim)

        grads = []
        for p in pos:
            unflatten_weights(model, p)
            model.zero_grad()
            preds = model(X_train).squeeze()
            loss  = criterion(preds, y_train.squeeze())
            loss.backward()
            grads.append(flatten_gradients(model))
        grads = np.array(grads)

        vel = (
            w * vel
            + c1 * r1 * (pbest_pos - pos)
            + c2 * r2 * (gbest_pos - pos)
            - alpha * grads
        )

        pos = pos + vel
        if bounds is not None:
            pos = np.clip(pos, low, high)

        fitness = eval_fitness(pos)

        better = fitness < pbest_score
        pbest_pos[better]   = pos[better]
        pbest_score[better] = fitness[better]

        idx = np.argmin(pbest_score)
        if pbest_score[idx] < gbest_score:
            gbest_score = pbest_score[idx]
            gbest_pos   = pbest_pos[idx].copy()

        if verbose and it % 10 == 0:
            print(f"Iter {it:3d}/{n_iters}, gbest MSE = {gbest_score:.6f}")

    return gbest_score, gbest_pos

In [ ]:
# import torch

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# options = {"c1": 0.5, "c2": 0.3, "w": 0.9}

# results = {}
# for d, group in district_groups.items():
#     # 1) Prepare sequences and scaler
#     X_np, y_np, scaler = make_sequences(group, seq_len=10)

#     # 2) Init model on device and compute parameter‐space dim
#     model = LSTMRegressor(n_features=X_np.shape[-1]).to(device)
#     dim   = sum(p.numel() for p in model.parameters())

#     # 3) Convert data to torch tensors
#     Xb = torch.tensor(X_np, dtype=torch.float32).to(device)
#     yb = torch.tensor(y_np, dtype=torch.float32).to(device)

#     # 4) Hybrid PSO+gradient optimization
#     best_cost, best_pos = hybrid_pso_optimize(
#         model, Xb, yb, dim,
#         n_particles=30,
#         n_iters=100,
#         w=options["w"],
#         c1=options["c1"],
#         c2=options["c2"],
#         alpha=0.01,         # gradient step size
#         bounds=(-1, 1),
#         verbose=False
#     )

#     # 5) Load the best‐found weights
#     unflatten_weights(model, best_pos)

#     # 6) Store results
#     results[d] = {"model": model, "scaler": scaler, "train_mse": best_cost}
#     print(f"District {d}: train MSE = {best_cost:.4f}")

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

options = {"c1": 0.5, "c2": 0.3, "w": 0.9}

results = {}
for d, group in district_groups.items():
    # 1) build full sequences
    X, y, scaler = make_sequences(group, seq_len=10)

    # 2) split into train & test by time
    split_idx = int(len(X) * 0.8)     # first 80% for training
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]

    # 3) convert to torch and move to device
    Xb_train = torch.tensor(X_train, dtype=torch.float32, device=device)
    yb_train = torch.tensor(y_train, dtype=torch.float32, device=device)
    Xb_test  = torch.tensor(X_test,  dtype=torch.float32, device=device)
    yb_test  = torch.tensor(y_test,  dtype=torch.float32, device=device)

    # 4) create & train model via hybrid PSO
    model = LSTMRegressor(n_features=X.shape[-1]).to(device)
    dim   = sum(p.numel() for p in model.parameters())
    best_cost, best_pos = hybrid_pso_optimize(
        model, Xb_train, yb_train, dim,
        n_particles=30, n_iters=100,
        w=0.9, c1=0.5, c2=0.3, alpha=0.01,
        bounds=(-1,1), verbose=True
    )
    unflatten_weights(model, best_pos)

    # 5) evaluate on test set
    model.eval()
    with torch.no_grad():
        y_pred_scaled = model(Xb_test).cpu().numpy()
    # inverse‐scale predictions & ground truth
    y_pred = scaler.inverse_transform(y_pred_scaled)
    y_true = scaler.inverse_transform(y_test)
    test_mse = ((y_pred - y_true)**2).mean()

    # 6) store and report
    results[d] = {
      "model": model,
      "scaler": scaler,
      "train_mse": best_cost,
      "test_mse": test_mse
    }
    print(f"District {d}: train MSE = {best_cost:.4f}, test MSE = {test_mse:.4f}")

In [ ]:
# options = {"c1": 0.5, "c2": 0.3, "w": 0.9}

# results = {}
# for d, group in district_groups.items():
#     X, y, scaler = make_sequences(group, seq_len=10)
#     fitness, dim, model = make_fitness(X, y)

#     # library implemented PSO
#     opt = GlobalBestPSO(n_particles=30, dimensions=dim, options=options)
#     best_cost, best_pos = opt.optimize(fitness, iters=100, verbose=False)

#     # hand made PSO
#     # best_cost, best_pos = pso_optimize(
#     #     fitness,
#     #     dim,
#     #     n_particles=30,
#     #     n_iters=100,
#     #     w=0.9,
#     #     c1=0.5,
#     #     c2=0.3,
#     #     bounds=(-1, 1),    # optional: clamp weights to [-1, 1]
#     #     verbose=True
#     # )
    
#     unflatten_weights(model, best_pos)
#     results[d] = {"model": model, "scaler": scaler, "train_mse": best_cost}
#     print(f"District {d}: train MSE = {best_cost:.4f}")

District 1: train MSE = 0.0101
District 2: train MSE = 0.0212
District 3: train MSE = 0.0169
District 4: train MSE = 0.0109
District 5: train MSE = 0.0179
District 6: train MSE = 0.0224
District 7: train MSE = 0.0100
District 8: train MSE = 0.0235
District 9: train MSE = 0.0192
District 10: train MSE = 0.0184
District 11: train MSE = 0.0254
District 12: train MSE = 0.0117
District 13: train MSE = 0.0164
District 15: train MSE = 0.0202
District 16: train MSE = 0.0281
District 17: train MSE = 0.0077
District 18: train MSE = 0.0141
District 19: train MSE = 0.0106
District 20: train MSE = 0.0296
District 21: train MSE = 0.0182
District 22: train MSE = 0.0244
District 23: train MSE = 0.0151
District 24: train MSE = 0.0152
District 25: train MSE = 0.0204
District 26: train MSE = 0.0191
District 27: train MSE = 0.0181
District 28: train MSE = 0.0073
District 29: train MSE = 0.0082
District 30: train MSE = 0.0212
District 31: train MSE = 0.0141
District 32: train MSE = 0.0107
District 33: trai